# Section 4 - R comme RAM

Cette section s'appuie largement sur le très bon article d'Advanced R : http://adv-r.had.co.nz/memory.html
<br>
# <span style="color: steelblue">3. Comprendre le fonctionnement de la mémoire en R</span>

R est construit pour charger les données calculées en mémoire vive (RAM). 
Si cela a permis une reactivité superieure au chargement en mémoire morte (ROM), cela limite egalement intraitablement la quantité de données à la mémoire physiquement installée dans l'ordinateur.  
A moins d'utiliser des solutions "out-of-memory" (type RevoScaleR dans Revolution R Enterprise), R alloue systematiquement la mémoire pour chaque objet de la session de travail.  
Il est donc necessaire de faire attention à la gestion de la mémoire pour éviter le fatidique "cannot allocate memory"





# <span style="font-family:Calibri">3.1 En savoir plus sur le systeme</span>

Avant toute discussion sur la mémoire, il est necessaire de connaitre quelques elements sur le systeme (ie notre PC !)  
(pour info, quelques elements sur les variables d'environnement ici : https://stat.ethz.ch/R-manual/R-devel/library/base/html/EnvVar.html)

In [1]:
version
Sys.getenv("R_ARCH")      #.i386     : 32bits / x64      : 64bits
.Machine$sizeof.pointer   # 4bytes   : 32bits / 8bytes   : 64bits

               _                           
platform       x86_64-w64-mingw32          
arch           x86_64                      
os             mingw32                     
system         x86_64, mingw32             
status                                     
major          3                           
minor          2.3                         
year           2015                        
month          12                          
day            10                          
svn rev        69752                       
language       R                           
version.string R version 3.2.3 (2015-12-10)
nickname       Wooden Christmas-Tree       

[1] "/x64"

[1] 8

En 32 bits, on a un bus de 32 "cellules" qui peuvent prendre la valeur 0 ou 1, soit 2^32 codes differents que le système peut adresser.  
Or 2 puissance 32 = 4 294 967 296 adresses différentes, ce qui correspond à 4 Go de RAM.  
Donc un processeur 32 bits ne peut pas depasser les 4GO
Mais comme il a besoin d'adresser autre chose que la RAM (ressources carte-mère, cartes graphiques, ...), il ne reste qu'environ 3 Go de RAM adressable suivant la configuration du PC


En 64 bits , le processeur peut adresser 4Go², soit prêt de 2*10^20 adresses differentes (ce qui ne signifie pas que le systeme d'exploitation les supportent).  
Windows 7 par exemple supporte jusqu'a 192 GB de RAM.  

Dans les faits, chez AXA, les ordinateurs standards disposent de 4Go de RAM, 16Go pour les postes developpeurs. Les serveurs, quant à eux,sont souvent par défaut limités a 2Go par user.

# <span style="font-family:Calibri">3.2. Memory.limit et object_size</span>


In [2]:
memory.limit()

[1] 3583

In [3]:
## on peut essayer de jouer au malin...
memory.limit(size=5000)

ERROR: Error in memory.limit(size = 5000): ne soyez pas stupide ! Votre machine a une limite de mémoire adressable de 4 Go


Pour comprendre la gestion de la mémoire dans R, et la place que prennent les objets, on peut utiliser pryr::object_size(). <br>
<b>object_size()</b> : poids des objets en mémoire (en bytes)

In [ ]:
library(pryr)
object_size(1:10)
object_size(mean)
object_size(mtcars)

Point important : un objet ne rend la mémoire qu'il occupe qu'une fois que <u>plus aucun autre objet n'y fait reference</u>.  
Prenons l'exemple ci dessous en utilisant <b>mem_change()</b>, qui indique le differentiel d'espace occupé suite a l'action réalisée

In [ ]:
# Creation d'un objet x, et copie dans y
mem_change(x <- 1:1e8)
mem_change(y <- x)



In [ ]:
# Pas d'impact de la suppression de x car y pointe toujours dessus
mem_change(rm(x))


In [ ]:
# Maintenant que l'on supprime y, la mémoire est rendue
mem_change(rm(y))


# <span style="font-family:Calibri">3.3. Garbage Collection, ncells et vcells</span> 

En R, la mémoire peut etre caracterisée suivant 2 dimensions : 
- la mémoire allouée pour les vecteurs et les tableaux (Vcells)
- celle allouée aux objets comme les listes (Ncells)

en invoquant la fonction gc() (Garbage Collection), on peut voir ces résultats : 

In [4]:
(g <- gc(verbose = TRUE))

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,323540.0,8.7,592000.0,15.9,460000.0,12.3
Vcells,354750.0,2.8,786432.0,6.0,678032.0,5.2


 - Les <b>Ncells</b> prennent 28 bytes/cellule sur les systemes 32-bit (le double sur les systemes 64 bits)  
 - les <b>VCells</b> prennent 8 bytes/cellule quel que soit le systeme
 
 - la colonne <b>"used"</b> indique le nombre de cellules allouées (et le nbe correspondant de Mb, arrondi a 0.1Mb superieur)
 - la colonne <b>"gc trigger"</b> mentionne a quel moment la collecte sera lancée par R.
 - la colonne <b>“max used”</b> rappelle l'espace maximum utilisé depuis la dernière collecte /le dernier lancement du moteur R

In [ ]:
## Recalcul du nombre de cellules utilisées : 
round((g[1,1]*28)/1024^2, 2)

La collecte est automatiquement faite par R lorsque un objet n'est plus utilisé, i.e. plus aucun element ne pointe vers cet objet.  

Néanmoins, si on souhaite rendre de l'espace à un instant t: <b>gc(reset = TRUE, verbose = TRUE)</b>

# <span style="font-family:Calibri">3.4. multilevelPSA::lsos()</span> 

Cette fonction retourne egalement la taille des objets en mémoire de façon synthetique et user friendly.  
J'aurais plutot tendance à la conseiller.

In [ ]:
#install.packages("multilevelPSA", repos = "http://cran.us.r-project.org")
library(multilevelPSA, warn.conflicts = FALSE, verbose = FALSE)
lsos()

## 2.5.2. cannot allocate vector of ...

Grand classique sous R, qu'on ne sait pas forcément (ni toujours..) résoudre.  
Cette erreur souligne l'incapacité de R a allouer un vecteur/objet de la taille indiquée.  
A noter que dans un environnement a 32-bit, il se peut qu'il y ait suffisamment de mémoire libre, mais pas forcement de suffisamment grand blocs contigus d'adresse mémoire pour le mapper.

# <span style="font-family:Calibri">3.5. Autres pistes</span>
### Package filehash

Pour ceux qui connaissent les cours du John Hopkins Institute sur Coursera, notons l'existence du package filehash() developpé par Roger Peng.  
Celui permet de sauver des objets sur le disque et de rappeler automatiquement si necessaire.  



###  Pré-allouer la mémoire.

There are a whole family of functions in R called functionals, or apply functions, which take vectors (or matrices, or lists) of values and apply arbitrary functions to each. Because these can use arbitrary functions, they are NOT compiled. Functionals mostly are written in pure R, and they speed up code only in certain cases.

One operation that is slow in R, and somewhat slow in all languages, is memory allocation. So one of the slower ways to write a for loop is to resize a vector repeatedly, so that R has to re-allocate memory repeatedly, like this:

j <- 1
for (i in 1:10) {
    j[i] = 10
}
Here, in each repetition of the for loop, R has to re-size the vector and re-allocate memory. It has to find the vector in memory, create a new vector that will fit more data, copy the old data over, insert the new data, and erase the old vector. This can get very slow as vectors get big.

If one pre-allocates a vector that fits all the values, R doesn’t have to re-allocate memory each iteration, and the results can be much faster. Here’s how you’d do that for the above case:

j <- rep(NA, 10)
for (i in 1:10) {
    j[i] = 10
}